In [2]:
from datetime import datetime
from time import sleep

import numpy as np
import pandas as pd
import requests
import bs4
from selenium import webdriver

phantomjs_path = '/Users/atakata/Documents/phantomjs-2.1.1-macosx/bin/phantomjs'

In [2]:
url_2016_primary = 'http://www.politico.com/mapdata-2016/2016-election/primary/results/map/president/'
url_2016 = 'http://www.politico.com/2016-election/results/map/president/'

Gather The Data
Scrape the 2012 and 2016 state-county election results data from Politico, the 2008 data from
the NYC website, and the 2004 and 2000 data from Wiki. For each year, put your results in a
elections/YYYY folder, in the form of a .csv file containing the following columns:
* state, county: state and county name
* 1nd, 2nd, 3rd: the names of the top, second and third candidate (nan if no third candidate)
* votes1, votes2, votes3: the number of votes for the top, second and third candidate
* pct1, pct2, pct3: percentage votes of the top, second and third candidate
* party1, party2, party3: party of the top 3 candidates e.g. D / R / O (democrats, republicans or other)

In [101]:
browser = webdriver.PhantomJS(executable_path=phantomjs_path)
browser.set_window_size(1400,1000)

In [103]:
def clean_party(x):
    if 'democrat' in x.lower():
        return 'D'
    elif 'republican' in x.lower():
        return 'R'
    else:
        return 'O'

# 2016 General

soup = bs4.BeautifulSoup(requests.get(url_2016).content, 'html5lib')
state_data = soup.find_all('article', {'class': 'timeline-group'})

det_result_links = []
for s in state_data:
    state = s.find('a').text.strip()
    elections = s.find_all('li')
    for e in elections:
        det_result_links.append(e.find('a').get('href'))

data = []
for href in det_result_links:
    state = href.split('/')[-2].replace('-', ' ').title()
    print(state)
    
    browser.get(href)
    sleep(1)
    for _ in range(10):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.1)
#     soup = bs4.BeautifulSoup(requests.get(href).content, 'html5lib')
    soup = bs4.BeautifulSoup(browser.page_source, 'html5lib')

    county_elections = soup.find('section', {'class': 'content-group election-results'}).\
                    find_all('article', {'class': 'results-group'})    
    for ce in county_elections:
        county = ce.find('div', {'class': 'title'}).text
        try:
            results = ce.find('table', {'class': 'results-table'}).find_all('tr')[:3]
        except AttributeError:
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            sleep(0.5)
            results = ce.find('table', {'class': 'results-table'}).find_all('tr')[:3]
        
        data_row = np.array([None]*14)
        data_row[[0,1]] = state, county
        for ind, r in enumerate(results):
            party = clean_party(r.get('class'))
#             party = clean_party(r.find('span', {'class': 'name-combo'}).find('abbr').get('title'))
            name = ' '.join([x for x in list(r.find('span', {'class': 'name-combo'}).children) if isinstance(x, str)]).strip().upper()
            pct_votes = float(r.find('td', {'class': 'results-percentage'}).text.replace('%', '')) / 100
            votes = int(r.find('td', {'class': 'results-popular'}).text.replace(',', ''))
            
            data_row[list(range(2+ind, 14, 3))] = (name, votes, pct_votes, party)
        data.append(data_row)
            
#     break

data_df = pd.DataFrame(columns=('state', 'county', '1st', '2nd', '3rd', 'votes1', 'votes2', 'votes3',
                               'pct1', 'pct2', 'pct3', 'party1', 'party2', 'party3'),
                      data=data)
data_df.to_csv('elections/2016/general.csv')

In [139]:
del browser

In [140]:
browser = webdriver.PhantomJS(executable_path=phantomjs_path)
browser.set_window_size(1400,1000)

In [145]:
# 2016 Primary

# soup = bs4.BeautifulSoup(requests.get(url_2016).content, 'html5lib')
soup = bs4.BeautifulSoup(requests.get(url_2016_primary).content, 'html5lib')
state_data = soup.find_all('article', {'class': 'timeline-group'})

det_result_links = []
for s in state_data:
    state = s.find('a').text.strip()
    elections = s.find_all('li')
    for e in elections:
        det_result_links.append(e.find('a').get('href'))

data = []
for href in det_result_links:
    state = href.split('/')[-2].replace('-', ' ').title()
    print(state)
    
    browser.get(href)
    sleep(0.5)
    for _ in range(10):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.1)
#     soup = bs4.BeautifulSoup(requests.get(href).content, 'html5lib')
    soup = bs4.BeautifulSoup(browser.page_source, 'html5lib')

    county_elections = soup.find('section', {'class': 'content-group election-results'}).\
                    find_all('article', {'class': 'results-group'})    
    for ce in county_elections:
        county = ce.find('div', {'class': 'title'}).text
        results_tables = ce.find_all('table', {'class': 'results-table'})
        for rt in results_tables:
            try:
                results = rt.find_all('tr')[:3]
            except AttributeError:
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                sleep(0.5)
                results = rt.find_all('tr')[:3]
            if len(results) == 0:
                continue
            
            data_row = np.array([None]*14)
            data_row[[0,1]] = state, county
            for ind, r in enumerate(results):
                party = clean_party(r.get('class')[0])
    #             party = clean_party(r.find('span', {'class': 'name-combo'}).find('abbr').get('title'))
                name = ' '.join([x for x in list(r.find('span', {'class': 'name-combo'}).children) if isinstance(x, str)]).strip().upper()
                try:
                    pct_votes = float(r.find('td', {'class': 'results-percentage'}).text.replace('%', '')) / 100
                except AttributeError:
                    pct_votes = np.nan
                try:
                    votes = int(r.find('td', {'class': 'results-popular'}).text.replace(',', ''))
                except AttributeError:
                    votes = np.nan
                    
                data_row[list(range(2+ind, 14, 3))] = (name, votes, pct_votes, party)
            data.append(data_row)

Alabama
Alaska
Arizona


KeyboardInterrupt: 

In [142]:
data_df = pd.DataFrame(columns=('state', 'county', '1st', '2nd', '3rd', 'votes1', 'votes2', 'votes3',
                               'pct1', 'pct2', 'pct3', 'party1', 'party2', 'party3'),
                      data=data)

In [144]:
data_df.to_csv('elections/2016/primary.csv')

In [146]:
url_2012 = 'http://www.politico.com/story/2012/11/election-results-2012-by-state-083320'

In [158]:
soup = bs4.BeautifulSoup(requests.get(url_2012).content, 'html5lib')
toc = soup.find('div', {'class': 'story-text'})

In [232]:
links = sorted(list(set([x.get('href') for x in toc.find_all('a') if x.text == 'President'])))

In [204]:
def clean_party(x):
    if 'gop' in x.lower():
        return 'R'
    elif 'dem' in x.lower():
        return 'D'
    else:
        return 'O'

In [253]:
states = [x.text.strip() for x in toc.find_all('a') if x.get('href') and x.get('href').startswith('#') 
             and len(x.get('href')) == 3]

In [255]:
base_2012_url = 'http://www.politico.com/2012-election/results/president/{}'

In [269]:
data = []
for state in states:
    url = base_2012_url.format(state.lower().replace(' ', '-'))
    soup = bs4.BeautifulSoup(requests.get(url).content, 'html5lib')
    print(state)
    results_tables = soup.find('div', {'id': 'mainContent'}).find_all('tbody')
    if len(results_tables) > 1:
        results_tables = results_tables[1:]
    for rt in results_tables:
        results = rt.find_all('tr')[:3]
        try:
            county = ' '.join([x for x in list(rt.find('th', {'class': 'results-county'}).children) if isinstance(x, str)]).strip()        
        except AttributeError:
            county = state
        data_row = np.array([None]*14)
        data_row[[0,1]] = state, county
        for ind, r in enumerate(results):
            name = ' '.join([x for x in list(r.find('th', {'class': 'results-candidate'}).children) if isinstance(x, str)]).split('(')[0].strip().upper()
            party = clean_party(r.find('td', {'class': 'results-party'}).text)
            pct_votes = float(r.find('td', {'class': 'results-percentage'}).text.replace('%', '')) / 100
            votes = int(r.find('td', {'class': 'results-popular'}).text.replace(',', ''))
            data_row[list(range(2+ind, 14, 3))] = (name, votes, pct_votes, party)
        data.append(data_row)

Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
District of Columbia
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming


In [270]:
data_df = pd.DataFrame(columns=('state', 'county', '1st', '2nd', '3rd', 'votes1', 'votes2', 'votes3',
                               'pct1', 'pct2', 'pct3', 'party1', 'party2', 'party3'),
                      data=data)

In [276]:
data_df.to_csv('elections/2012/general.csv')

In [3]:
data_df = pd.read_csv('elections/2012/general.csv')